# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.112152e+02     1.457562e+00
 * time: 0.06360006332397461
     1     1.121972e+01     8.817408e-01
 * time: 1.3191859722137451
     2    -1.138178e+01     9.566399e-01
 * time: 1.4049968719482422
     3    -3.406165e+01     7.866613e-01
 * time: 1.4836318492889404
     4    -4.770789e+01     6.198698e-01
 * time: 1.5740978717803955
     5    -5.699553e+01     2.642592e-01
 * time: 1.6622259616851807
     6    -5.986243e+01     1.796988e-01
 * time: 1.7209508419036865
     7    -6.091721e+01     5.186016e-02
 * time: 1.7801399230957031
     8    -6.130446e+01     1.060200e-01
 * time: 1.847442865371704
     9    -6.160061e+01     5.465584e-02
 * time: 1.9060750007629395
    10    -6.180895e+01     3.531519e-02
 * time: 1.9652018547058105
    11    -6.197464e+01     2.047100e-02
 * time: 2.028594970703125
    12    -6.204557e+01     1.688063e-02
 * time: 2.086103916168213
    13    -6.212442e+01     1.820483e-02
 * time: 2.143148899078

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557661
    AtomicNonlocal      14.8522632
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666460946
